# Notebook setup

In [29]:
# imports
import sys
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pdb
import pandas as pd
from functools import reduce as rd
from sklearn.decomposition import PCA
%reload_ext autoreload
%autoreload 2

In [2]:
!mkdir final # make a directory in Local Disk
!mkdir data

In [31]:
# set up constants for ease of directory use
FINAL_DIR = 'gs://final-bucket-jiaying'
DATA_DIR = f'{BUCKET_DIR}/data'
#!gsutil -m cp data/iris_full.csv {IRIS_DATA_WITH_LABELS}

#  EDA

## Glimpse of the data

In [32]:
# read 6 data from google cloud

# trianing2_fdwind.csv
!gsutil cat {DATA_DIR}/training2_fdwind.csv | head # there are some missing for temperature 

#=================
#!TODO 1: figure out how to deal with missing of 2 columns: bearing, knots, temperature
#=================

fbwindairportid,ordinal,bearing,knots,temperature
243222,0,90,12,
243222,1,90,12,
243222,2,90,12,
243222,3,80,12,
243222,4,60,12,15
243222,5,50,10,11
243222,6,60,8,8
243222,7,60,7,3
243222,8,80,5,-4


In [33]:
!gsutil cat {DATA_DIR}/training2_fdwindairport.csv | head

fbwindairportid,fbwindreportid,airportcode
243222,3018,JON
243223,3018,KWA
243224,3018,MAJ
243225,3018,MDY
243226,3018,PPG
243227,3018,TTK
243228,3018,AWK
243229,3018,GRO
243230,3018,GSN


In [34]:
!gsutil cat {DATA_DIR}/training2_fdwindaltitude.csv | head

fbwindreportid,ordinal,altitude
2632,0,1000
2632,1,1500
2632,2,2000
2632,3,3000
2632,4,6000
2632,5,9000
2632,6,12000
2632,7,15000
2632,8,18000


In [35]:
!gsutil cat {DATA_DIR}/training2_asdiflightplan.csv | head

#=================
#! TODO 2: For the timestamp, we might want to seperate them into 6 column: year, month, day, hour, minute, second
#          But before doing that, we need to consider whether we want to do it 
#          a. before join, or after join
#          b. before creating the predicting label or after
#=================

id,update_time_utc,flighthistory_id,departure_airport,arrival_airport,aircraft_id,legacy_route,original_departure_utc,estimated_departure_utc,original_arrival_utc,estimated_arrival_utc
114294485,2013-08-21 13:45:45+00,306898411,SAC,OAK,AMF287,SAC..OAK,2013-08-22 06:45:00+00,2013-08-22 06:55:00+00,2013-08-22 07:15:00+00,2013-08-22 07:20:00+00
114332979,2013-08-21 18:58:29+00,306610801,CGI,STL,KAP409,CGI..STL/0055,2013-08-21 11:00:00+00,2013-08-21 12:42:00+00,2013-08-21 12:05:00+00,2013-08-21 13:23:00+00
114775397,2013-08-22 18:54:16+00,306904458,FSD,MBG,CCQ600,FSD..MBG,2013-08-22 10:30:00+00,2013-08-22 11:54:00+00,2013-08-22 11:40:00+00,2013-08-22 13:03:00+00
114850278,2013-08-22 22:14:00+00,306995156,YKM,MWH,CFS797,YKM..MWH,2013-08-22 15:00:00+00,2013-08-22 15:14:00+00,2013-08-22 15:50:00+00,2013-08-22 15:40:00+00
117989171,2013-08-30 06:59:01+00,307685375,VIS,FAT,AMF238,VIS..FAT/0015,2013-08-30 02:40:00+00,2013-08-30 01:11:00+00,2013-08-30 03:00:00+00,2013-08-30 01:25:00+00
118889106,

In [36]:
!gsutil cat {DATA_DIR}/training2_asdiposition.csv | head

#=================
#!TODO 3: We are still debating whether using this data.
#         Its size is over 7GB which will be a good practice for us to deal with it in this class.
#         But, so far we have not find any good ways to combine with other datasets that we choosed.
#         We will try to think more about it. Hope we could use it eventually.
#=================

id,received,callsign,altitude,ground_speed,latitude_degrees,longitude_degrees,flighthistory_id
108923402,2013-09-09 16:27:54+00,AWE404,37000,454,39.3300018310547,-91.7300033569336,308536896
108923403,2013-09-09 16:27:54+00,EJA955,43000,496,39.1300010681152,-95.9700012207031,308759273
108923404,2013-09-09 16:27:54+00,JBU223,36000,467,37.1699981689453,-98.9800033569336,308510756
108923405,2013-09-09 16:27:54+00,UAL436,37000,474,38.9199981689453,-97.5299987792969,308546747
108923406,2013-09-09 16:28:00+00,SKW6407,33000,451,43.1699981689453,-115.269996643066,308543571
108923407,2013-09-09 16:27:55+00,UAL1680,7400,238,33.3199996948242,-112.180000305176,308507714
108923412,2013-09-09 16:27:57+00,OPT304,45000,387,35.6500015258789,-80.7200012207031,308758984
108923413,2013-09-09 16:27:58+00,SKW5490,27000,483,35.6500015258789,-119.550003051758,308543959
108923414,2013-09-09 16:27:57+00,DAL564,25700,477,33.6500015258789,-85.6699981689453,308472934


In [37]:
!gsutil cat {DATA_DIR}/training2_flighthistory.csv | head # missing some published_arrival 

#=================
#!TODO 4: !!!!!
#         Since in the data, there is no column to direct tell whether the flights are deplayed or not.
#         So we want to join this table with training2_asdiflightplan.csv, then we will have two columns:
#         original_arrival_utc, and published_arrival.
#         How we understand it is that,
#         1. original_arrival_utc, is the time the flights are scheduled to be arrived;
#         2. published_arrival, is the time that the flights are actually arrived.
#         To compare their difference, if published_arrival is **greater/latter** than original_arrival_utc
#=================

id,airline_code,airline_icao_code,flight_number,departure_airport_code,departure_airport_icao_code,arrival_airport_code,arrival_airport_icao_code,published_departure,published_arrival,scheduled_gate_departure,actual_gate_departure,scheduled_gate_arrival,actual_gate_arrival,scheduled_runway_departure,actual_runway_departure,scheduled_runway_arrival,actual_runway_arrival,creator_code,scheduled_air_time,scheduled_block_time,departure_airport_timezone_offset,arrival_airport_timezone_offset,scheduled_aircraft_type,actual_aircraft_type,icao_aircraft_type_actual
307025132,G4,AAY,565,EUG,KEUG,LAS,KLAS,2013-08-26 02:25:00+00,2013-08-26 04:20:00+00,2013-08-26 02:25:00+00,,2013-08-26 04:20:00+00,,,,,,I,,115,-7,-7,M80,,
307089377,9K,KAP,3312,BOS,KBOS,MVY,KMVY,,,2013-08-23 17:20:00+00,,2013-08-23 17:57:00+00,,,,,,D,,37,-4,-4,,,
307391933,AA,,23,DEN,KDEN,DFW,KDFW,,,2013-08-26 15:25:00+00,,2013-08-26 17:20:00+00,,2013-08-26 15:40:00+00,,2013-08-26 17:31:00+00,,D,111,115,-6,-5,,,
306201343,ZW,AWI,19,D

## Read data in

In [30]:
# start a spark session
from pyspark.sql import SparkSession

app_name = "final_project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()

sc = spark.sparkContext

In [62]:
# read from local disk:
# Problem: For example the flighthistory, somehow the data that is in local disk is not able to load fully.
#          This is also an issue for all 6 data that we chosen. When we read it, it either shows less rows, or less columns
# Solution: Read direct from google cloud bucket using spark might solve the problem.

# fdwind = pd.read_csv('data/training2_fdwind.csv',header=None)
# fdwind.columns = "fbwindairportid,ordinal,bearing,knots,temperatur".split(',')

# fdwindairport = pd.read_csv('data/training2_fdwindairport.csv', header=None)
# fdwindairport.columns = "fbwindairportid,fbwindreportid,airportcode".split(',')

# fdwindaltitude = pd.read_csv('data/training2_fdwindaltitude.csv', header=None)
# fdwindaltitude.columns = "fbwindreportid,ordinal,altitude".split(',')

# flighthistory = pd.read_csv('data/training2_flighthistory.csv', header=None)
# flighthistory.columns = "id,airline_code,airline_icao_code,flight_number,departure_airport_code,departure_airport_icao_code,arrival_airport_code,arrival_airport_icao_code,published_departure,published_arrival,scheduled_gate_departure,actual_gate_departure,scheduled_gate_arrival,actual_gate_arrival,scheduled_runway_departure,actual_runway_departure,scheduled_runway_arrival,actual_runway_arrival,creator_code,scheduled_air_time,scheduled_block_time,departure_airport_timezone_offset,arrival_airport_timezone_offset,scheduled_aircraft_type,actual_aircraft_type,icao_aircraft_type_actual".split(',')

# asdiflightplan = pd.read_csv('data/training2_asdiflightplan.csv', header=None)
# asdiflightplan.columns = "id,update_time_utc,flighthistory_id,departure_airport,arrival_airport,aircraft_id,legacy_route,original_departure_utc,estimated_departure_utc,original_arrival_utc,estimated_arrival_utc".split(',')

# asdiposition = pd.read_csv('data/training2_asdiposition.csv', header=None)
# asdiposition.columns = "id,received,callsign,altitude,ground_speed,latitude_degrees,longitude_degrees,flighthistory_id".split(",")

In [76]:
# read from google cloud using spark

fdwind = spark.read.csv('gs://final-bucket-jiaying/data/training2_fdwind.csv',inferSchema=True,header=True)
fdwind.printSchema()
fdwind.show()

#=================
#!TODO 5: The nullability is not right in this data, also same for the other data we chose.
#         Later we want to change the nullability since the IDs cannot be null.  
#=================

root
 |-- fbwindairportid: integer (nullable = true)
 |-- ordinal: integer (nullable = true)
 |-- bearing: integer (nullable = true)
 |-- knots: integer (nullable = true)
 |-- temperature: integer (nullable = true)

+---------------+-------+-------+-----+-----------+
|fbwindairportid|ordinal|bearing|knots|temperature|
+---------------+-------+-------+-----+-----------+
|         243222|      0|     90|   12|       null|
|         243222|      1|     90|   12|       null|
|         243222|      2|     90|   12|       null|
|         243222|      3|     80|   12|       null|
|         243222|      4|     60|   12|         15|
|         243222|      5|     50|   10|         11|
|         243222|      6|     60|    8|          8|
|         243222|      7|     60|    7|          3|
|         243222|      8|     80|    5|         -4|
|         243222|      9|    110|    5|        -17|
|         243223|      0|    130|    8|       null|
|         243223|      1|    130|    8|       null|
|   

In [77]:
fdwindairport = spark.read.csv('gs://final-bucket-jiaying/data/training2_fdwindairport.csv',inferSchema=True,header=True)
fdwindairport.printSchema()
fdwindairport.show()

root
 |-- fbwindairportid: integer (nullable = true)
 |-- fbwindreportid: integer (nullable = true)
 |-- airportcode: string (nullable = true)

+---------------+--------------+-----------+
|fbwindairportid|fbwindreportid|airportcode|
+---------------+--------------+-----------+
|         243222|          3018|        JON|
|         243223|          3018|        KWA|
|         243224|          3018|        MAJ|
|         243225|          3018|        MDY|
|         243226|          3018|        PPG|
|         243227|          3018|        TTK|
|         243228|          3018|        AWK|
|         243229|          3018|        GRO|
|         243230|          3018|        GSN|
|         243231|          3018|        TNI|
|         243232|          3018|        GUM|
|         243233|          3018|        TKK|
|         243234|          3018|        PNI|
|         243235|          3018|        ROR|
|         243236|          3018|        T11|
|         243237|          3019|        JON|
|

In [78]:
fdwindaltitude = spark.read.csv('gs://final-bucket-jiaying/data/training2_fdwindaltitude.csv',inferSchema=True,header=True)
fdwindaltitude.printSchema()
fdwindaltitude.show()

root
 |-- fbwindreportid: integer (nullable = true)
 |-- ordinal: integer (nullable = true)
 |-- altitude: integer (nullable = true)

+--------------+-------+--------+
|fbwindreportid|ordinal|altitude|
+--------------+-------+--------+
|          2632|      0|    1000|
|          2632|      1|    1500|
|          2632|      2|    2000|
|          2632|      3|    3000|
|          2632|      4|    6000|
|          2632|      5|    9000|
|          2632|      6|   12000|
|          2632|      7|   15000|
|          2632|      8|   18000|
|          2632|      9|   24000|
|          2633|      0|    1000|
|          2633|      1|    1500|
|          2633|      2|    2000|
|          2633|      3|    3000|
|          2633|      4|    6000|
|          2633|      5|    9000|
|          2633|      6|   12000|
|          2633|      7|   15000|
|          2633|      8|   18000|
|          2633|      9|   24000|
+--------------+-------+--------+
only showing top 20 rows



In [79]:
flighthistory = spark.read.csv('gs://final-bucket-jiaying/data/training2_flighthistory.csv',inferSchema=True,header=True)
flighthistory.printSchema()
flighthistory.show()

#=================
#!TODO 6: Here we see the column like "published_departure", which is a timestamp, shows as sting.
#         So later when we read in the data, we might want to read it in as a timestamp.
#         Also, same for the other data.
#=================

root
 |-- id: integer (nullable = true)
 |-- airline_code: string (nullable = true)
 |-- airline_icao_code: string (nullable = true)
 |-- flight_number: integer (nullable = true)
 |-- departure_airport_code: string (nullable = true)
 |-- departure_airport_icao_code: string (nullable = true)
 |-- arrival_airport_code: string (nullable = true)
 |-- arrival_airport_icao_code: string (nullable = true)
 |-- published_departure: string (nullable = true)
 |-- published_arrival: string (nullable = true)
 |-- scheduled_gate_departure: string (nullable = true)
 |-- actual_gate_departure: string (nullable = true)
 |-- scheduled_gate_arrival: string (nullable = true)
 |-- actual_gate_arrival: string (nullable = true)
 |-- scheduled_runway_departure: string (nullable = true)
 |-- actual_runway_departure: string (nullable = true)
 |-- scheduled_runway_arrival: string (nullable = true)
 |-- actual_runway_arrival: string (nullable = true)
 |-- creator_code: string (nullable = true)
 |-- scheduled_air_

In [80]:
asdiflightplan = spark.read.csv('gs://final-bucket-jiaying/data/training2_asdiflightplan.csv',inferSchema=True,header=True)
asdiflightplan.printSchema()
asdiflightplan.show()

root
 |-- id: integer (nullable = true)
 |-- update_time_utc: string (nullable = true)
 |-- flighthistory_id: integer (nullable = true)
 |-- departure_airport: string (nullable = true)
 |-- arrival_airport: string (nullable = true)
 |-- aircraft_id: string (nullable = true)
 |-- legacy_route: string (nullable = true)
 |-- original_departure_utc: string (nullable = true)
 |-- estimated_departure_utc: string (nullable = true)
 |-- original_arrival_utc: string (nullable = true)
 |-- estimated_arrival_utc: string (nullable = true)

+---------+--------------------+----------------+-----------------+---------------+-----------+--------------------+----------------------+-----------------------+--------------------+---------------------+
|       id|     update_time_utc|flighthistory_id|departure_airport|arrival_airport|aircraft_id|        legacy_route|original_departure_utc|estimated_departure_utc|original_arrival_utc|estimated_arrival_utc|
+---------+--------------------+----------------+---

In [81]:
asdiposition = spark.read.csv('gs://final-bucket-jiaying/data/training2_asdiposition.csv',inferSchema=True,header=True)
asdiposition.printSchema()
asdiposition.show()

root
 |-- id: integer (nullable = true)
 |-- received: string (nullable = true)
 |-- callsign: string (nullable = true)
 |-- altitude: integer (nullable = true)
 |-- ground_speed: integer (nullable = true)
 |-- latitude_degrees: double (nullable = true)
 |-- longitude_degrees: double (nullable = true)
 |-- flighthistory_id: integer (nullable = true)

+---------+--------------------+--------+--------+------------+----------------+-----------------+----------------+
|       id|            received|callsign|altitude|ground_speed|latitude_degrees|longitude_degrees|flighthistory_id|
+---------+--------------------+--------+--------+------------+----------------+-----------------+----------------+
|108923402|2013-09-09 16:27:...|  AWE404|   37000|         454|39.3300018310547|-91.7300033569336|       308536896|
|108923403|2013-09-09 16:27:...|  EJA955|   43000|         496|39.1300010681152|-95.9700012207031|       308759273|
|108923404|2013-09-09 16:27:...|  JBU223|   36000|         467|37.1

## Statistical summary

In [83]:
fdwind.describe().toPandas()

# For fdwind, the count of "temperature" is down about 22000 compare to the total number of records.
#=================
#!TODO 7: There are two strategies for dealing with the missingness of "temperature"
#         1. Drop rows whenever their temperature is missing;
#         2. More complicated way: Find out which one is missing. 
#            For the same "fbwindairportid", if all the temperature is missing => drop all rows for that "fbwindairportid"
#                                            if some are missing, not all => find the "ordinal", fill the temperature based on the ordinal information
#            If we decided to fill in the temperature after join with the "training2_fdwindaltitude.csv",
#                                            we could calculate the missing temperature by the altitude based on the knowledge:
#                                           "For every 100-meter increase in elevation, the average temperature decreases by 0.7°C."
#=================
# Link: https://www.ces.fau.edu/nasa/module-3/why-does-temperature-vary/elevation.php#:~:text=Temperature%20normally%20decreases%20as%20elevation,decreases%20by%200.7%C2%B0C.

# More things to discuss:
# Is the temperature in unit celsius (C) or fahrenheit (F)?
# Based on my research, I will say it is in Celsius. In the data documents, there is not no clue for which one it is.
# Here is the link that help me made this decision. (https://www.ncbi.nlm.nih.gov/books/NBK207472/#:~:text=As%20shown%20in%20Figure%202,F)%20(ASHRAE%201999a).)

,summary,fbwindairportid,ordinal,bearing,knots,temperature
0,count,270723,270723,270723,270723,248497
1,mean,229483.74540028,3.4396227878680423,266.01241859760717,26.758613047284495,-27.026346394523877
2,stddev,10291.085514174132,2.4051620607615654,189.54975667154352,20.339123917879157,24.422557303710434
3,min,211666,0,10,0,-69
4,max,247311,9,990,154,29


In [84]:
fdwindairport.describe().toPandas() # no missing

,summary,fbwindairportid,fbwindreportid,airportcode
0,count,35646,35646,35646
1,mean,229488.5,2853.8625091174326,None
2,stddev,10290.258184322,126.22468163039072,None
3,min,211666,2632,2XG
4,max,247311,3071,ZUN


In [85]:
fdwindaltitude.describe().toPandas() # no missing

# Is the unit of altitude in feet (ft) or meter (m)?
# Base on my research, I think it is in feet.
# Here is the link that help me made this decision. (https://www.ncbi.nlm.nih.gov/books/NBK207472/#:~:text=As%20shown%20in%20Figure%202,F)%20(ASHRAE%201999a).)

,summary,fbwindreportid,ordinal,altitude
0,count,3870,3870,3870
1,mean,2851.1501291989666,4.02015503875969,17279.328165374678
2,stddev,127.14545551589654,2.6995891968470778,13682.584409244842
3,min,2632,0,1000
4,max,3071,9,53000


In [86]:
flighthistory.describe().toPandas() 

# missing all: "actual_aircraft_type"
# missing some: "airline_icao_code", "published_departure"

#=================
#!TODO 10: check "flight_number", whether there is any meaning for 1 and 9999. 
#          My concern is, since the number is so nice for min and max. I doubt that those number might have some meaning.
#=================

,summary,id,airline_code,airline_icao_code,flight_number,departure_airport_code,departure_airport_icao_code,arrival_airport_code,arrival_airport_icao_code,published_departure,...,scheduled_runway_arrival,actual_runway_arrival,creator_code,scheduled_air_time,scheduled_block_time,departure_airport_timezone_offset,arrival_airport_timezone_offset,scheduled_aircraft_type,actual_aircraft_type,icao_aircraft_type_actual
0,count,711576,711576,710064,711576,711576,711576,711576,711576,611317,...,680194,660365,711576,680194,620382,711576,711576,611317,0,648924
1,mean,3.072983116636185E8,None,None,2589.9427861535523,None,None,None,None,None,...,None,None,None,94.44022440656636,129.125506542743,-4.9947187651073115,-4.998542671478521,541.6872984125268,None,None
2,stddev,809621.9702907128,None,None,2073.3864941937923,None,None,None,None,None,...,None,None,None,61.16069340888028,95.90125519272513,1.1281392217587365,1.1286867870501252,210.3955832719113,None,None
3,min,305796740,1I,AAH,1,AAF,KAAF,AAF,KAAF,2013-08-14 09:00:00+00,...,2013-08-14 09:22:00+00,2013-08-14 08:47:00+00,A,-65,1,-7,-7,319,None,A124
4,max,308963256,ZW,XOJ,9999,ZZV,KZZV,ZZV,KZZV,2013-09-11 08:41:00+00,...,2013-09-11 12:15:00+00,2013-09-11 14:07:00+00,I,1422,1670,-4,-4,SH6,None,WB57


In [91]:
from pyspark.sql.functions import col

for col_name in flighthistory.columns:
    col_stats = flighthistory.select(col(col_name)).describe().toPandas()
    print(f"Statistics for column '{col_name}':")
    print(col_stats)
    
# missing all: 
# "actual_aircraft_type": the "icao_aircraft_type_actual" can be used to represent the same information
#=================
#!TODO 8: detele the whole column, "actual_aircraft_type" 
#=================

# missing some: 
## IMPORTATNT:
# "published_arrival":
#=================
#!TODO 9: !!!!!
#         Since we need "published_arrival" to decide whether the flight is delayed or not, it is important.
#         So for those rows that has "published_arrival" missing => drop them
#=================

# missing some: [We will keep those columns in the way that they originaly are]
## Not Important:
# "airline_icao_code": not important since airline_code has no missing
# "published_departure": not important since we only need arrival time to decide whether the flight is delayed or not 
# scheduled_gate_departure, actual_gate_departure, scheduled_gate_arrival, actual_gate_arrival, 
#    scheduled_runway_departure, actual_runway_departure, scheduled_runway_arrival, actual_runway_arrival
#    scheduled_air_time, scheduled_block_time: n
# "scheduled_aircraft_type": not too important
# "icao_aircraft_type_actual": not too important

#=================
#!TODO 11: check "scheduled_air_time", the min is negative. 
#          Base on its defination: Computed number of minutes from the scheduled runway departure to the scheduled runway arrival.
#               it should be all positive. 
#          (Since all the time that showed in the data is in Pacific Time Zone, we do not need consider time difference.)
#=================

Statistics for column 'id':
  summary                   id
0   count               711576
1    mean  3.072983116636185E8
2  stddev    809621.9702907128
3     min            305796740
4     max            308963256
Statistics for column 'airline_code':
  summary airline_code
0   count       711576
1    mean         None
2  stddev         None
3     min           1I
4     max           ZW
Statistics for column 'airline_icao_code':
  summary airline_icao_code
0   count            710064
1    mean              None
2  stddev              None
3     min               AAH
4     max               XOJ
Statistics for column 'flight_number':
  summary       flight_number
0   count              711576
1    mean  2589.9427861535523
2  stddev  2073.3864941937923
3     min                   1
4     max                9999
Statistics for column 'departure_airport_code':
  summary departure_airport_code
0   count                 711576
1    mean                   None
2  stddev                   None


In [92]:
asdiflightplan.describe().toPandas()

# Missing: "departure_airport" [not imporant since we are more care about arrival airport]

,summary,id,update_time_utc,flighthistory_id,departure_airport,arrival_airport,aircraft_id,legacy_route,original_departure_utc,estimated_departure_utc,original_arrival_utc,estimated_arrival_utc
0,count,7954958,7954958,7954958,7954765,7954957,7954958,7954958,7954958,7954958,7954958,7954958
1,mean,1.2202349433752649E8,None,3.0730841514516985E8,None,21.0,None,None,None,None,None,None
2,stddev,7150581.1692793695,None,793899.6939766831,None,0.0,None,None,None,None,None,None
3,min,114132776,2013-08-14 16:00:01+00,305796740,07FA,07FA,AAH320,07FA..DHP..KMIA/0008,1970-01-01 01:00:00+00,2013-08-13 22:08:00+00,1970-01-01 01:00:00+00,2013-08-14 03:37:00+00
4,max,136407537,2013-09-11 20:35:39+00,308963256,ZSPD,ZZV,XOJ795,ZSPD.LAM22D.LAMEN.A593.SADLI.A593.POTET.A593.F...,2013-09-11 11:49:00+00,2013-09-11 12:50:00+00,2013-09-11 12:35:00+00,2013-09-11 14:03:00+00


In [ ]:
asdiposition.describe().toPandas()

# "altitude", only miss one row, not a big problem.

,summary,id,received,callsign,altitude,ground_speed,latitude_degrees,longitude_degrees,flighthistory_id
0,count,81406063,81406063,81406063,81406062,81406063,81406063,81406063,81406063
1,mean,9.258699936393818E7,None,None,25467.264152404277,386.82306031677274,37.87955559436281,-94.82404213820048,3.072929107970119E8
2,stddev,6.698126499659612E7,None,None,12466.599530340098,97.39953787539626,4.371310714793561,14.361708387842867,806346.3601494217
3,min,5,2013-08-14 08:30:56+00,AAH320,-1,0,15.4167,-177.4667,305840319
4,max,215416430,2013-09-11 15:34:57+00,XOJ795,99900,999,68.75,178.1333,308963256


# Outcome (evaluation metric and the target)

In [ ]:
# # Task 2
# from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Example true labels and predicted labels
# y_true = ...
# y_pred = ...

# # Calculating evaluation metrics
# precision = precision_score(y_true, y_pred)
# recall = recall_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)

# # Printing evaluation metrics
# print(f'Precision: {precision:.2f}')
# print(f'Recall: {recall:.2f}')
# print(f'F1 Score: {f1:.2f}')

# # Generating the confusion matrix
# conf_matrix = confusion_matrix(y_true, y_pred)

# # Plotting the confusion matrix using seaborn
# sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.show()

# Ingest the data files

In [2]:
# read dataset
# import pandas as pd
# file_path = ‘path/to//csvfile.csv’
# df = pd.read_csv(file_path)
# print(df.head())
# check datasets
# column_a = df[‘column_name_a’]
# columns_subset = df[[‘column_name_a’, ‘column_name_b’]]
# df[‘new_column’] = df[‘column_name_a’] + df[‘column_name_b’]
# summary_stats = df[‘column_name’].describe()
# grouped_data = df.groupby(‘column_name’).mean()

# Summarized relevant datasets

- fdwind_all.csv: <br>

outer join ( training2_fdwindaltitude.csv & training2_fdwindairport.csv ) <br>
outer join ( 1 & training2_fdwind.csv )
with column “fbwindairportid”, “ordinal”

- asdi_flighthistory.csv: <br>

outer join ( training2_flighthistory.csv & training2_asdiflightplan.csv )
	with column “id” == “flighthistory_id”
    
[still under consideration] outer join ( 1 & training2_asdiposition.csv) with column “flighthistory_id”

- final_flight_full.csv: <br>

right join ( fdwind_all.csv & asdi_flighthistory.csv )
	with column “airportcode” == “arrival_airport_code”
    
right join ( 1 & asdi_flighthistory.csv )
	with column “airportcode” == “departure_airport_code”

# Check Points for Joining

In [1]:
# # Task 5
# # Join the dataframes on a key
# df_joined = pd.merge(df1, df2, on='key', suffixes=('_df1', '_df2'))

# # Checkpoint the joined dataframe to a CSV file
# checkpoint_filepath = 'joined_data_checkpoint.csv'
# df_joined.to_csv(checkpoint_filepath, index=False)

# # Checkpoint the joined dataframe to a Parquet file
# checkpoint_filepath = 'joined_data_checkpoint.parquet'
# df_joined.to_parquet(checkpoint_filepath, index=False)

# Split the data

In [3]:
# from sklearn.model_selection import train_test_split
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# # Pandas DataFrame
# # target_column is the name of target variable column
# # Separating features and the target variable
# X = df.drop(columns=[target_column])
# y = df[target_column]
# # Splitting the dataset into training set and the remainder (test set + validation set)
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
# # Splitting the remainder into the test set and validation set
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
# # Initialize the scaler
# scaler = StandardScaler()
# # Fit the scaler using only the training set
# scaler.fit(X_train)
# # Transform all datasets using the statistics from the training set
# X_train_scaled = scaler.transform(X_train)
# X_val_scaled = scaler.transform(X_val)
# X_test_scaled = scaler.transform(X_test)